<a href="https://colab.research.google.com/github/Tanushree-233/Twitter_Sentiment_Analysis/blob/main/Twitter_Sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spacy emoji scikit-learn pandas
!python -m spacy download en_core_web_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 92.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import pandas as pd
import re
import emoji
import spacy

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
train_df = pd.read_csv("twitter_training.csv", header=None)
val_df = pd.read_csv("twitter_validation.csv", header=None)

data = pd.concat([train_df, val_df], ignore_index=True)

In [ ]:
data.columns = ["id", "entity", "sentiment", "text"]
data = data[["text", "sentiment"]]

In [ ]:
data["sentiment"] = data["sentiment"].str.lower()
data = data[data["sentiment"] != "irrelevant"]

In [ ]:
emoji_sentiment = {
    "😀": 1, "😃": 1, "😄": 1, "😊": 1, "😍": 1, "❤️": 1, "🔥": 1,
    "😡": -1, "😠": -1, "😢": -1, "😭": -1, "💔": -1
}

In [ ]:
def emoji_score(text):
    score = 0
    for char in text:
        if char in emoji_sentiment:
            score += emoji_sentiment[char]
    return score

In [ ]:
def clean_tweet(text):
    text = text.lower()
    score = emoji_score(text)

    # Remove URLs, mentions, hashtags
    text = re.sub(r"http\S+|www\S+|@\w+|#\w+", "", text)

    # Remove emojis (already scored)
    text = emoji.replace_emoji(text, replace="")

    doc = nlp(text)

    tokens = [
        token.lemma_
        for token in doc
        if token.is_alpha and not token.is_stop
    ]

    cleaned_text = " ".join(tokens)

    # Add emoji sentiment signal
    if score > 0:
        cleaned_text += " positiveemoji"
    elif score < 0:
        cleaned_text += " negativeemoji"

    return cleaned_text

In [ ]:
data["text"] = data["text"].fillna("")
data["clean_text"] = data["text"].apply(clean_tweet)

In [ ]:
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(data["clean_text"])
y = data["sentiment"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

In [ ]:
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.8185792349726776
              precision    recall  f1-score   support

    negative       0.78      0.90      0.84      1020
     neutral       0.89      0.65      0.75       743
    positive       0.82      0.86      0.84       982

    accuracy                           0.82      2745
   macro avg       0.83      0.80      0.81      2745
weighted avg       0.83      0.82      0.82      2745



In [ ]:
def predict_sentiment(tweet):
    cleaned = clean_tweet(tweet)
    vector = vectorizer.transform([cleaned])
    return model.predict(vector)[0]

In [ ]:
print(predict_sentiment("I love this phone 😍🔥"))
print(predict_sentiment("Worst service ever 😡💔"))
print(predict_sentiment("Delivery was okay"))
print(predict_sentiment("I don't like it"))
print(predict_sentiment("It was too good"))

positive
negative
negative
negative
positive
